# 필수 개인 과제
- LLM 과 RAG를 활용하여 AI 챗봇을 구현해보자.

### 필요한 라이브러리들을 불러온다.

In [1]:
import os   # 운영체제와 상호작용하게 해주는 표준 라이브러리
from dotenv import load_dotenv  # .env 파일에서 환경 변수를 불러올 수 있게 해주는 라이브러리

from langchain_openai import ChatOpenAI # LangChain 라이브러리에서 OpenAI의 챗봇 모델을 사용하기 위한 코드. 즉, OpenAI의 챗 모델을 사용하여 언어 모델을 초기화하기 위한 코드
from langchain_core.messages import HumanMessage    # LangChain 라이브러리에서 HumanMessage 클래스를 임포트하는 코드

from langchain_community.document_loaders import PyPDFLoader  # LangChain 라이브러리에서 PDF 파일을 로드하는 데 사용하는 코드

from langchain.text_splitter import CharacterTextSplitter   # 텍스트를 여러 덩어리로 분할하는 기능
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 긴 텍스트를 의미가 끊기지 않도록 일정한 길이의 조각으로 나누는 역할

from langchain_openai import OpenAIEmbeddings   # langchain_openai 라이브러리에서 OpenAIEmbeddings 클래스를 불러온다.

import faiss    # 벡터 검색을 위한 라이브러리
from langchain_community.vectorstores import FAISS  # # langchain_community에서 FAISS 벡터스토어 클래스를 임포트한다.

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# LangChain의 ChatPromptTemplate와 RunnablePassthrough 클래스를 불러온다.

_**HumanMessage**_ 클래스는 LangChain의 메시지 처리 시스템에서 사용자 메시지를 나타내는 데 사용된다.   
즉, LangChain에서 인간의 메시지를 모델로 전달하는 데 필요한 형태로 포맷팅해준다.   
LangChain은 대화형 AI 시스템을 구축할 때 여러 종류의 메시지를 처리하는데, HumanMessage는 사람이 보낸 메시지를 모델에 전달할 때 사용된다.

_**PyPDFLoader**_ 는 PDF 파일을 로드하여 텍스트를 추출하는 클래스다.   
PyPDFLoader를 langchain.document_loaders에서 임포트하는 것이 더 이상 권장되지 않기 때문에  
**langchain_community.document_loaders**에서 임포트해야 한다.

_**FAISS**_ (Facebook AI Similarity Search)는 고속 유사도 검색을 위한 라이브러리다. 주로 벡터 데이터베이스에서 빠르게 유사한 항목을 검색하는 데 사용된다.   
_**langchain_community.vectorstores**_ 는 Langchain의 벡터스토어 모듈로, 다양한 벡터 데이터베이스에 데이터를 저장하고 검색할 수 있는 기능을 제공한다. 여기서는 FAISS 벡터스토어를 사용하여 문서들을 벡터화하고 저장하려고 하고 있다.

_**ChatPromptTemplate**_ 은 대화형 프롬프트 템플릿을 정의하는 클래스다. 시스템 메시지와 사용자 메시지를 설정하고, context와 question을 템플릿에서 사용할 수 있도록 정의한다.

## 1. 사용환경 준비

In [2]:
# .env 파일 로드
load_dotenv()   # .env 파일을 로드하여 환경 변수들을 설정한다.

True

In [3]:
# .env 파일에서 api 키 가져오기
API_KEY = os.getenv('sparta_api_key')

_**os.getenv**_ 는 환경 변수에서 값을 가져오는 역할을 한다. 여기서는 api 키 값을 가져왔다.  
즉, .env 파일에 저장된 'sparta_api_key'라는 환경 변수의 값을 가져왔다.

In [4]:
# API 키가 잘 로드되었는지 확인
if API_KEY is None:
    raise ValueError("API key is missing from .env file")
# print(API_KEY)

_**print(API_KEY)**_ 를 통해 API_KEY를 직접 확인해볼 수도 있다.

In [5]:
# 환경 변수에 API 키 설정
os.environ['OPENAI_API_KEY'] = API_KEY

이렇게 설정하면 openai 라이브러리가 환경 변수를 사용한다.  
openai.api_key = API_KEY ==> 이건 옛날 버전이니 위의 코드를 사용하자.  
이제 OpenAI 라이브러리가 자동으로 환경 변수를 사용하여 API 키를 설정한다.  
ChatOpenAI 모델을 초기화 할 때 API 키를 별도로 전달할 필요가 없다.

```python
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")
```
이 코드를 통해 api key가 잘 작동하는 지 확인할 수도 있다.

## 2. 모델 로드하기

In [6]:
# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini") # ChatOpenAI 객체를 초기화하여 사용할 모델을 설정한다.

## 3. 문서 로드하기

In [7]:
# PyPDFLoader 인스턴스를 생성하고 PDF 파일을 로드할 준비를 한다. 파일의 경로 입력
loader = PyPDFLoader("./[2024 한권으로 ok 주식과 세금].pdf")

In [8]:
# PDF에서 텍스트 추출. 페이지 별 문서 로드
docs = loader.load()    # loader.load()를 호출하여 PDF 파일의 텍스트를 페이지별로 불러온다.

아래 코드를 통해 추출된 텍스트를 확인해볼 수 있다.
```python
for doc in docs:
    print(doc)
```

## 4. 문서 청크로 나누기

### **CharacterTextSplitter**
먼저 CharacterTextSplitter에 대해 알아보자. 이는 텍스트를 일정한 문자 단위로 나누기 위한 도구다. 주로 자연어 처리를 할 때 긴 문서를 다루는 데 사용된다. LangChain 라이브러리에서 제공하며, 텍스트를 일정한 크기로 조각내서 대규모 언어 모델이나 요약, 질문응답 작업에 적합하게 만들어 준다.

주요 특징을 살펴보자.

최대 길이 설정: 각 텍스트 조각이 가지는 최대 길이를 지정할 수 있다. 이를 통해 모델이 부담 없이 처리할 수 있는 크기로 텍스트를 나눌 수 있다.
중복 포함 옵션: 조각 간에 중복되는 텍스트를 추가할 수도 있는데, 이는 문맥을 유지하고 문장의 연속성을 확보하는 데 도움을 준다.
구분자 설정: 특정 구분자를 기준으로 텍스트를 나눌 수 있다. 예를 들어, 문단을 구분할 때 줄바꿈 문자(\n)나 공백을 기준으로 나눌 수 있다.

즉, 설정한 최대 길이 안에서 단순히 문자 수를 기준으로 텍스트를 나누고, 중복을 포함하여 조각을 자연스럽게 연결할 수 있도록 해준다.

### **RecursiveCharacterTextSplitter**
다음으로 RecursiveCharacterTextSplitter에 대해 알아보자. 이는 CharacterTextSplitter보다 더 유연하고 똑똑하게 텍스트를 나누기 위해 사용된다. 긴 텍스트를 일정한 길이로 나누는 점에서는 CharacterTextSplitter와 비슷하지만, 분할할 때 여러 기준을 사용하여 최대한 문장이나 문단의 흐름을 해치지 않도록 텍스트를 조각낸다는 점에서 다르다. recursive는 '순환의'라는 뜻을 가진다. 내가 해석하기로는 흐름을 해치지 않는 선에서 문단, 문장, 단어 순으로 구분자를 시도하기 때문에 순환한다고 하는 것 같다.

정리하면, 이 방식은 각 조각이 설정된 길이에 맞게 최대한 의미를 유지하며 나뉘고, 중복 구간이 포함되어 문맥을 자연스럽게 연결할 수 있게 해준다.

1. CharacterTextSplitter

In [9]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",   # 두 개의 개행 문자를 구분자로 사용
    chunk_size=100, # 최대 100자씩 나눈다.
    chunk_overlap=10,   # 각 조각에 앞뒤로 5자의 중복을 포함한다.
    length_function=len,    # 길이를 계산할 때 문자 수(len)를 기준으로 한다.
    is_separator_regex=False,   # 구분자를 단순 문자열로 처리
)

splits = text_splitter.split_documents(docs)

print(splits[:10]) # 청킹된 내용 상위 10개 출력

[Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.\n최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  \n쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  \n이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  \n부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 \n발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를\n기대합니다.\n2024.  5\n국세청 자산과세국장\n머리말'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 3}, page_content='본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한\n것으로 모든 법령규정을 담고 있지는 않습니다.\n또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생\n할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인\n하시기 바랍니다.\n본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  \n유예된 법령 등은 반영되어 있지 않습니다.\n본 책자에 표기된 

2. RecursiveCharacterTextSplitter

In [10]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 최대 100자씩 나눈다.
    chunk_overlap=10,   # 각 조각에 앞뒤로 5자의 중복을 포함한다.
    length_function=len,    # 길이를 계산할 때 문자 수(len)를 기준으로 한다.
    is_separator_regex=False,   # 구분자를 단순 문자열로 처리
)

splits_RCT = recursive_text_splitter.split_documents(docs)

print(splits_RCT[:10])  # 청킹된 내용 상위 10개 출력

[Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록

#### 파라미터를 살펴보자.
- **separator** : 텍스트를 나눌 때 사용할 구분자다. 여기서는 엔터 2번(\n\n)으로 구분하여 텍스트를 나누게 설정했다. 구분자를 정규 표현식으로 사용할지 안 할지는 *is_separator_regex* 파라미터로 결정된다.
- **chunk_size** : 나눠진 각 chunk의 최대 길이를 지정한다. 여기서는 100으로 설정했는데, 각 chunk의 최대 길이가 100문자라는 뜻이다.
- **chunk_overlap** : 각 chunk 간에 겹치는 부분의 길이를 설정한다. 여기서는 10으로 설정했으니, 앞 chunk의 마지막 10문자가 다음 chunk의 시작 부분에도 포함될 것이다. 이렇게 겹치는 부분을 통해 문맥을 유지할 수 있다.
- **length_function** : chunk의 길이를 계산할 때 사용할 함수다. 기본값은 len으로, chunk의 *문자의 개수*를 센다. 필요에 따라 다른 함수를 지정해 길이를 다르게 정의할 수도 있다.
- **is_separator_regex** : *separator* 파라미터가 정규 표현식인지 여부를 설정한다. **False**로 설정하면 *separator*는 단순한 문자열로 취급되고, **True**로 설정하면 정규 표현식으로 인식된다.

과제에서는 한 가지 청킹 방식만 사용하면 된다.   
나는 기본적으로 길이 단위로 나누는 _**CharacterTextSplitter**_ 청킹 방식을 선택했다.

## 5. 벡터 임베딩 생성
OpenAI 모델을 사용했기 때문에 OpenAIEmbeddings를 이용해 텍스트를 벡터로 변환할 벡터 임베딩을 생성했다.

In [11]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

'text-embedding-ada-002' 모델을 사용하여 텍스트 데이터를 임베딩 벡터로 변환한다.   
이 모델은 텍스트 데이터를 고차원 벡터로 변환하는 데 사용된다.   
자연어 처리(NLP) 작업에서 텍스트를 벡터 공간으로 변환하여, 유사도 검색, 클러스터링, 분류 등 다양한 작업에 활용된다.

## 6. 벡터 스토어 생성

앞서 만든 벡터 임베딩과 청크된 문서를 활용하여 FAISS 벡터 스토어를 생성했다.

In [12]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

_**FAISS.from_documents**_ 는 splits_RCT라는 문서 목록을 받아들여, 각 문서를 임베딩 모델(embeddings)을 사용하여 벡터로 변환한 후, 이 벡터들을 FAISS 벡터스토어에 저장하는 역할을 한다.   
_**splits_RCT**_ 는 우리가 pdf 문서들을 chunk 단위로 나눈 데이터다. 이 데이터를 벡터로 변환하여 검색할 수 있게 documents에 저장한다.   
_**embedding**_ 에는 텍스트 임베딩 모델을 지정한다. 여기서는 OpenAI의 text-embedding-ada-002 모델을 사용하여 문서 벡터를 생성한다.

## 7. FAISS를 Retriever로 변환

RAG 체인에서 사용할 수 있도록 FAISS를 retriever로 변환하자.

In [13]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

_**RAG**_ (Retrieval-Augmented Generation) 체인에서 FAISS를 retriever로 변환하는 이유는 검색 기반 텍스트 생성을 효율적으로 수행하기 위해서다. RAG 체인에서는 **검색**과 **생성**의 두 단계가 중요한 역할을 한다. 이를 통해 모델은 주어진 질문에 대한 답변을 더 잘 생성할 수 있도록 외부 지식을 활용한다.

_**as_retriever()**_ 메서드는 vectorstore를 retriever로 변환하는 역할을 한다. 이 메서드는 벡터 데이터베이스에서 검색을 수행할 수 있도록 변환해 준다.  
_**search_type="similarity"**_ 에 의해 유사도 검색이 실행되고, _**{"k": 1}**_ 설정에 따라 가장 유사한 문서 1개만 검색해서 반환한다.

## 8. 프롬프트 템플릿 정의

프롬프트 템플릿을 정의하자.

In [14]:
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

_**system**_ 메시지를 통해 모델에게 주어진 문맥 내에서만 질문에 답하라는 지시를 내렸다.   
_**user**_ 메시지에는 실제 _**context**_ 와 _**question**_ 이 채워질 수 있도록 변수를 포함한 프롬프트 템플릿을 생성했다.   
따라서 {context}와 {question}은 나중에 실제 사용자가 제공한 값으로 대체된다.   
   


## 9. RAG 체인 구성

LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성해보자.

In [15]:
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        # 부모 클래스의 invoke 메서드를 호출하여 처리된 결과를 받아온다.
        output = super().invoke(*args, **kwargs)
        # 처리된 결과를 출력하여 디버깅 용도로 확인한다.
        print("Debug Output:", output)
        # 처리된 결과를 그대로 반환한다.
        return output

RunnablePassthrough 클래스를 상속받은 _**DebugPassThrough**_ 클래스를 정의한다.   
이 클래스는 입력된 데이터를 그대로 전달하면서, 중간 결과를 디버깅 용도로 print한다.

In [16]:
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수도 받을 수 있도록 설정
        # context의 각 문서를 텍스트로 결합한다.
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        # 결합된 텍스트와 사용자 질문을 함께 반환한다.
        return {"context": context_text, "question": inputs["question"]}

문서 리스트를 텍스트로 변환하는 _**ContextToText**_ 클래스를 정의한다.   
RunnablePassthrough를 상속받은 또다른 클래스다.   
inputs["context"]로 전달된 문서 리스트를 하나의 텍스트로 결합하고, context와 question을 반환한다.

In [17]:
# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                  # retriever는 context를 가져오는 단계다.
    "question": DebugPassThrough()         # DebugPassThrough는 question을 그대로 전달하며 디버깅을 출력한다.
} | DebugPassThrough() | ContextToText() | contextual_prompt | model  # 각 단계에 디버깅과 텍스트 변환을 추가한 파이프라인

RAG 체인에서 각 단계에 DebugPassThrough를 추가했다.   
**rag_chain_debug** 는 질문 응답 시스템의 각 단계를 정의한 파이프라인이다.   
**retriever** 는 문서에서 관련된 context를 가져온다.
**DebugPassThrough** 는 사용자의 질문이 잘 전달되는지 확인하고, 디버깅 출력을 확인한다.    
이후 **ContextToText** 가 문서 리스트를 텍스트로 변환한다.   
마지막으로 **contextual_prompt** 로 위에서 정의한 템플릿을 사용하여 질문에 대한 답변을 생성하고, **model** 로 모델을 호출하여 답변을 생성한다. 즉, 프롬프트 템플릿을 사용하여 모델을 호출한다.

## 10. 챗봇 구동 확인

질문에 응답하는 챗봇을 구동하여 질문해보자.   

코드는 **while** 문을 통해 무한 루프를 시작한다. 이 루프는 사용자가 질문을 입력할 때마다 계속 반복된다. 루프를 종료하려면 break 명령어를 사용하거나 프로그램을 강제로 종료해야 한다.   
**query = input("질문을 입력하세요: ")** 는 input() 함수를 사용하여 사용자가 질문을 입력할 수 있게 한다. 이 입력값은 query 변수에 저장된다.   
**rag_chain_debug.invoke(query)** 를 호출하여, 사용자가 입력한 질문(query)을 rag_chain_debug 체인을 통해 처리한다. 이 체인은 여러 단계를 거쳐서 질문에 대한 답변을 생성한다.   
rag_chain_debug는 앞서 10. RAG 체인 구성에서 정의했다시피, 문서 검색, 질문 전달, 디버깅 출력, 텍스트 변환, 프롬프트 처리 등을 포함한 파이프라인으로 질문을 처리한다.   
**print("Final Response:")** 는 모델로부터 받은 최종 응답을 출력하기 전 "Final Response:"라는 메시지를 표시한다.   
**print(response.content)** 는 response는 처리된 결과이며, content는 모델의 응답을 포함하는 속성이다. 이 값을 출력하여 사용자가 입력한 질문에 대한 최종 응답을 표시한다.

In [19]:
while True:  
    # 사용자에게 질문을 입력하라는 메시지를 출력
    print("========================")
    
    # 사용자로부터 질문을 입력받음
    query = input("질문을 입력하세요 (종료하려면 'exit'를 입력하세요): ")

    # 'exit'를 입력하면 프로그램을 종료시킴
    if query.lower() == 'exit':
        print("프로그램을 종료합니다.")
        break
    
    # 'rag_chain_debug' 체인을 호출하여 질문을 처리하고 응답을 받음
    # 이 때, 'query'는 사용자가 입력한 질문이다.
    response = rag_chain_debug.invoke(query)  
    
    # 'Final Response:'라는 메시지를 출력하여 최종 응답을 나타냄
    print("Final Response:")
    
    # 'response.content'는 모델이 반환한 응답의 내용을 출력한다.
    print(response.content)

Debug Output: 주식에 대해 자세하게 설명해줘.
Debug Output: {'context': [Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 14}, page_content='3\n조금 더 알아보기\n주식이란 주식회사를 설립하거나 사업확장 등을 위해 필요한 자금을 조달할 때 투자자 \n에게 자금을 보탠 대가로 발행해 주는 증서로서 주식회사의 소유지분을 표시하는 단위 \n입니다.\n주식은 작은 금액의 단위 (1주당 100원 이상) 로 발행되는데, 이는 많은 사람이 자신의  \n사정에 맞게 투자할 수 있도록 하기 위한 것입니다. 결국 주식회사가 수많은 사람들로부터 \n자금을 제공받아 그것을 원천으로 기업활동을 계속할 수 있도록 해 주는 것이죠.\n주식을 소유한 사람을 주주라고 하고, 주주는 회사의 자본금 중 자신이 출자한 금액  \n만큼 회사의 주인이 되며, 출자지분에 비례하여 배당을 받게 됩니다. 회사 측면에서는 \n주주가 출자한 자금만큼 자기자본이 형성되고 만기의 개념이 없어 주식발행을 통해  \n조달한 자금은 되돌려줄 필요가 없습니다.\n뿐만 아니라 기업운영에서 발생하는 이익의 범위 내에서 배당금을 지급하면 되기  \n때문에 차입이나 채권발행에 비해 기업의 부담이 크게 줄어드는 이점이 있습니다. \n가. 주식의 개념\n주식이란 주식회사를 설립하거나 사업확장 등을 위해 필요한 자금을 \n조달할 때 투자자에게 자금을 보탠 대가로 발행해 주는 증서로서 주식 \n회사의 소유지분을 표시하는 단위입니다.\n주식은 주권(증서)에 액면가액이 기재되었는지 여부, 기명·날인 유무, \n재산적 내용,  의결권 유무, 상장 여부 등에 따라 여러 종류로 구분됩니다.\n01    주식의 개념과 종류가 궁금해요')], 'question': '주식에 대해 자세하게 설명해줘.'}
Final Response:
주식은 주식회사를 설립하거나 사업 확장을 위해 필요한 자금을 조달하기 위해 투자자에게 발행하는 증


같은 질문을 일반 chat gpt 혹은 Gemini에 질문해보고 답변을 비교해보자.

### **gpt-3.5-turbo**
##### 질문1 : 주식에 대해 자세하게 설명해줘. 
##### 대답1
주식은 기업의 소유권을 나타내는 증권으로, 기업이 발행한 주식을 구매한 사람은 해당 기업의 소유주가 됩니다. 주식을 보유한 사람은 기업의 이익을 나누는 권리와 주식 시장에서 주식을 매매할 수 있는 권리를 갖게 됩니다.

주식 시장은 기업이 주식을 발행하고 투자자들이 주식을 거래하는 시장을 말합니다. 시장에서 주식의 가격은 수요와 공급에 따라 변동하며, 이를 통해 기업의 가치나 경제 상황을 반영할 수 있습니다.

주식 시장은 주식을 통해 투자자들이 자본을 확보하고 수익을 창출하는 장소이기도 합니다. 주식 투자는 수익을 창출할 수 있는 기회를 제공하지만, 동시에 투자 금액을 잃을 수도 있는 위험이 따르는 투자 방법이기도 합니다.

주식 시장은 주식시장의 종류에 따라 국내 주식시장과 해외 주식시장으로 나눌 수 있으며, 투자자들은 자신의 투자 목표와 성향에 맞게 적합한 시장을 선택하여 투자할 수 있습니다.  

##### 질문2 : 상장주식 대주주에 대해서 자세히 설명하고, 50억원 이상으로 완화된 기준이 언제부터 적용되는 지 알려줘.
##### 대답2
상장주식 대주주란 일정 비율 이상의 주식을 보유하고 있는 주주를 말합니다. 대주주는 회사의 경영에 큰 영향력을 행사할 수 있기 때문에 주식시장에서 중요한 역할을 합니다. 대주주는 보통 주식회사법에서 정한 기준에 따라 결정되며, 대부분의 경우 10% 이상의 주식을 보유한 경우 대주주로 인정됩니다.

50억원 이상으로 완화된 기준은 2019년 3월 29일부터 적용되었습니다. 이 기준은 대주주의 지분 소유 비율이 10% 이상이면서, 지분의 시가평가액이 50억원 이상인 경우에 해당됩니다. 이 기준은 대주주의 지분 소유 비율이 낮아도 시가평가액이 높은 경우에 대주주로 인정되도록 완화된 조건입니다.

------------------------
### **Gemini**
##### 질문1 : 주식에 대해 자세하게 설명해줘.   
##### 대답1
**주식**

주식은 주식회사의 소유권을 대표하는 유가증권으로, 회사의 지분을 갖는 것을 의미합니다. 주식을 보유하면 회사의 수익과 자산에 대한 권리를 갖게 됩니다.

**주요 특징:**

* **소유권:** 주식은 회사의 소유권 증서입니다.
* **분배금:** 회사가 수익을 낼 경우 주주에게 분배금이 지급될 수 있습니다.
* **자본 이득:** 주식 가격이 상승하면 주주는 주식을 매도하여 자본 이득을 실현할 수 있습니다.
* **손실 위험:** 주식 가격은 변동될 수 있으며, 투자한 금액보다 손실을 입을 위험이 있습니다.

**주식 종류:**

**보통주:** 회사의 일반적인 소유권을 나타내며, 일반적으로 투표권이 있습니다.
**우선주:** 주식 가격에 관계없이 고정된 분배금을 지급받을 권리가 있습니다. 투표권이 없는 경우가 많습니다.

**주식 거래:**

주식은 증권 거래소에서 거래됩니다. 거래소는 매수자와 매도자를 연결하여 주식 가격을 결정하는 역할을 합니다.

**투자 목적:**

사람들이 주식에 투자하는 이유는 다음과 같습니다.

* **장기적인 성장:** 주식 시장은 역사적으로 다른 자산 클래스보다 높은 수익률을 제공해 왔습니다.
* **수익 창출:** 주식은 분배금을 지급하여 수익을 창출할 수 있습니다.
* **인플레이션 대응:** 분배금과 주식 가치 증가는 인플레이션에 맞서 보호막이 될 수 있습니다.
* **분산:** 주식 투자는 다른 자산과 포트폴리오를 분산시켜 위험을 줄이는 데 도움이 됩니다.

**주의 사항:**

* 주식 투자는 항상 위험이 따릅니다. 투자한 금액보다 손실을 입을 가능성이 있습니다.
* 주식 시장은 변동성이 있으므로 수익률은 보장되지 않습니다.
* 주식 투자를 하기 전에 투자 목표, 위험 허용 수준, 투자 시간 범위를 신중하게 고려하는 것이 중요합니다.

##### 질문2 : 상장주식 대주주에 대해서 자세히 설명하고, 50억원 이상으로 완화된 기준이 언제부터 적용되는 지 알려줘.
##### 대답2
**상장주식 대주주**

상장주식 대주주는 다음 기준을 충족하는 특정 주주를 말합니다.

* 상장주식의 소유 비율이 5% 이상 또는
* 상장주식의 소유 개수가 500만 주 이상

**대주주의 의무**

대주주는 다음과 같은 의무를 갖습니다.

* 주식 소유 현황을 공시 증권 감독원에 보고
* 주식 매수/매도에 대한 내부 거래 정보 공개
* 중요한 경영 결정에 대한 투표 및 참여
* 회사에 대한 충성 의무

**완화된 기준**

2023년 3월 1일부터 상장주식 대주주의 기준이 다음과 같이 완화되었습니다.

* 상장주식의 소유 비율 기준: 5%에서 **2%**로 낮아짐
* 상장주식의 소유 개수 기준: 500만 주에서 **200만 주**로 낮아짐

따라서, **2023년 3월 1일부터는 상장주식의 소유 비율이 2% 이상 또는 소유 개수가 200만 주 이상인 주주는 상장주식 대주주로 간주됩니다.**


### 왜 RAG이 필요한가?

주식에 대해 자세하게 설명해달라는 질문에는 gpt-3.5-turbo와 gemini 모두 일반적인 지식으로 잘 답변을 했다.   
주목할 부분은 '상장주식 대주주에 대해서 자세히 설명하고, 50억원 이상으로 완화된 기준이 언제부터 적용되는 지 알려줘.'라는 질문에 대한 답변이다.   
- **gpt-3.5-turbo**의 답변을 보면 할루시네이션이 발생한 것처럼 보이다.   
- **Gemini**도 정확한 시점을 대답하지 못했고, 관련된 설명을 늘어놓았다.   
- **RAG**를 활용해서 만든 챗봇은 vectorstore에 넣어둔 pdf 자료를 통해 정확한 시점을 말해줄 수 있었다.

RAG를 사용하면 LLM이 외부 데이터, 여기서는 pdf 파일을 활용해서, 더 정확한 답변을 생성할 수 있게 된다. 최신 정보나 특정한 데이터에 대해 외부 정보를 반영해서 답변을 생성하기 때문에 최신 정보에 대한 답변도 잘하고, 더 정확하게 답변할 수 있는 것이다.